# Titanic - Feature Engineering!
In this notebook, we will begin engineering features appropriate to build our machine learning model 

In [53]:
# Importing the necessary Python libraries
import pandas as pd
from category_encoders import one_hot
from sklearn.impute import SimpleImputer

In [2]:
# Loading in the training dataset
df_train = pd.read_csv('../data/train.csv')

## What We Did Last Time...

In [3]:
# Dropping unnecessary columns
df_train.drop(columns = ['PassengerId', 'Name', 'Ticket', 'Cabin'], inplace = True)

In [4]:
# Changing Sex (gender) to numerical values
df_train['Sex'].mask(df_train['Sex'] == 'male', 0, inplace = True)
df_train['Sex'].mask(df_train['Sex'] == 'female', 1, inplace = True)
df_train['Sex'] = df_train['Sex'].astype(int)

In [5]:
# Separating the supporting features (X) from the predictor feature (y)
X = df_train.drop(columns = ['Survived'])
y = df_train[['Survived']]

## Starting our Feature Engineering!

In [6]:
X.head()

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,3,0,22.0,1,0,7.2500,S
1,1,1,38.0,1,0,71.2833,C
2,3,1,26.0,0,0,7.9250,S
3,1,1,35.0,1,0,53.1000,S
4,3,0,35.0,0,0,8.0500,S


In [7]:
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Pclass    891 non-null    int64  
 1   Sex       891 non-null    int64  
 2   Age       714 non-null    float64
 3   SibSp     891 non-null    int64  
 4   Parch     891 non-null    int64  
 5   Fare      891 non-null    float64
 6   Embarked  889 non-null    object 
dtypes: float64(2), int64(4), object(1)
memory usage: 48.9+ KB


## Engineering the Embarked Column

In [8]:
# Checking the value counts on the Embarked column
X['Embarked'].value_counts()

S    644
C    168
Q     77
Name: Embarked, dtype: int64

In [40]:
# Instantiating the One Hot Encoder object
embarked_ohe_encoder = one_hot.OneHotEncoder(use_cat_names = True, handle_missing = 'S')

In [41]:
# Showing which "Embarked" records from the raw dataset are null
X[X['Embarked'].isnull()]

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
61,1,1,38.0,0,0,80.0,NaN
829,1,1,62.0,0,0,80.0,NaN


In [42]:
# Performing one hot encoding on the "Embarked" column
embarked_dummies = embarked_ohe_encoder.fit_transform(X['Embarked'])

/Users/dkhundley/opt/anaconda3/lib/python3.8/site-packages/category_encoders/utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):


In [43]:
# Viewing the "Embarked" dummies
embarked_dummies.tail()

,Embarked_S,Embarked_C,Embarked_Q,Embarked_nan
886,1,0,0,0
887,1,0,0,0
888,1,0,0,0
889,0,1,0,0
890,0,0,1,0


In [47]:
# Joining "Embarked" dummies back into X dataset
X = pd.concat([X, embarked_dummies], axis = 1)

In [51]:
# Getting rid of original "Embarked" column
X.drop(columns = ['Embarked'], inplace = True)

## Engineering the Age column

In [60]:
# Instantiating the "Age" imputer object
age_imputer = SimpleImputer(strategy = 'median')

In [63]:
# Fit / Transform the "Age" column with the imputer object
imputed_age = age_imputer.fit_transform(X['Age'].values.reshape(-1,1))

In [71]:
# Recreating the "Age" imputed feature as a Pandas DataFrame
df_imputed_age = pd.DataFrame(data = imputed_age, columns = ['age_imputed'])

In [72]:
# Joining imputed "Age" back into full X dataset
X = pd.concat([X, df_imputed_age], axis = 1)

In [74]:
# Getting rid of the origin "Age" column
X.drop(columns = ['Age'], inplace = True)

In [76]:
# Establishing our bins values and names
bin_labels = ['child', 'teen', 'young_adult', 'adult', 'elder']
bin_values = [-1, 12, 19, 30, 60, 100]

In [93]:
# Applying "Age" binning with Pandas cut
age_bins = pd.cut(X['age_imputed'], bins = bin_values, labels = bin_labels)
df_age_bins = pd.DataFrame(age_bins)

In [94]:
# Viewing the distribution of age bins
df_age_bins['age_imputed'].value_counts()

young_adult    422
adult          283
teen            95
child           69
elder           22
Name: age_imputed, dtype: int64

In [98]:
# Instantiating the "Age" One Hot Encoder (OHE) object
age_ohe_encoder = one_hot.OneHotEncoder(use_cat_names = True, handle_unknown = 'other')

In [99]:
# Fitting the age bins DataFrame to the OHE object
age_dummies = age_ohe_encoder.fit_transform(df_age_bins)

/Users/dkhundley/opt/anaconda3/lib/python3.8/site-packages/category_encoders/utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):


In [101]:
# Joining age_dummies back into full X dataset
X = pd.concat([X, age_dummies], axis = 1)

In [103]:
# Getting rid of the original "Age" (or actually, "age_imputed") column
X.drop(columns = ['age_imputed'], inplace = True)

In [105]:
# Viewing the final engineered features
X.head()

,Pclass,Sex,SibSp,Parch,Fare,Embarked_S,Embarked_C,Embarked_Q,Embarked_nan,age_imputed_child,age_imputed_teen,age_imputed_young_adult,age_imputed_adult,age_imputed_elder
0,3,0,1,0,7.2500,1,0,0,0,0,0,1,0,0
1,1,1,1,0,71.2833,0,1,0,0,0,0,0,1,0
2,3,1,0,0,7.9250,1,0,0,0,0,0,1,0,0
3,1,1,1,0,53.1000,1,0,0,0,0,0,0,1,0
4,3,0,0,0,8.0500,1,0,0,0,0,0,0,1,0


In [106]:
# Viewing information about the X DataFrame
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 14 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   Pclass                   891 non-null    int64  
 1   Sex                      891 non-null    int64  
 2   SibSp                    891 non-null    int64  
 3   Parch                    891 non-null    int64  
 4   Fare                     891 non-null    float64
 5   Embarked_S               891 non-null    int64  
 6   Embarked_C               891 non-null    int64  
 7   Embarked_Q               891 non-null    int64  
 8   Embarked_nan             891 non-null    int64  
 9   age_imputed_child        891 non-null    int64  
 10  age_imputed_teen         891 non-null    int64  
 11  age_imputed_young_adult  891 non-null    int64  
 12  age_imputed_adult        891 non-null    int64  
 13  age_imputed_elder        891 non-null    int64  
dtypes: float64(1), int64(13)
m

In [107]:
# Saving cleaned dataset to disk
X.to_csv('../data/cleaned_train.csv', index = False)
y.to_csv('../data/y.csv', index = False)